# Neural Network Foundations

In [1]:
import tensorflow as tf

In [2]:
tf.logging.set_verbosity(tf.logging.ERROR)

In [3]:
from keras.models import Sequential

Using TensorFlow backend.


In [4]:
from keras.layers import Dense

In [5]:
model = Sequential()
model.add(Dense(12, input_dim=8, kernel_initializer='random_uniform'))

The net is dense, meaning that each neuron in a layer is connected
to all neurons located in the previous layer and to all the neurons
in the following layer.

### Problems in Training the Perceptron and a Solution

We cannot simply use accuracy as the metric for optimization, because it is not continuous, we need a function that progressively changes from 0 to 1 with no discontinuity.

Mathematically, this means that we need
a continuous function that allows us to compute the derivative

Activation function
- sigmoid
- rectified linear unit (ReLU)

### A real example — recognizing handwritten digits

In [6]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD
from keras.utils import np_utils

In [7]:
np.random.seed(1671)

In [8]:
nb_epoch = 20
batch_size = 10000
verbose = 1
nb_classes = 10
optimizer = SGD()
validation_split = 0.2

In [9]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [10]:
X_train = X_train.reshape(60000, 784).astype('float32')
X_test = X_test.reshape(10000, 784).astype('float32')
X_train = X_train / 255
X_test = X_test / 255
print(X_train.shape[0], 'training samples')
print(X_test.shape[0], 'testing samples')

60000 training samples
10000 testing samples


In [11]:
y_train = np_utils.to_categorical(y_train, num_classes=10)
y_test = np_utils.to_categorical(y_test, num_classes=10)

In [12]:
model = Sequential()
model.add(Dense(nb_classes, input_shape=(784,)))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 10)                7850      
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
%%time
history = model.fit(X_train, y_train, epochs=nb_epoch, verbose=True, validation_split=validation_split)

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 2s 46us/step - loss: 0.6982 - acc: 0.8393 - val_loss: 0.4899 - val_acc: 0.8805
Epoch 2/20
48000/48000 [==============================] - 2s 43us/step - loss: 0.4786 - acc: 0.8755 - val_loss: 0.4128 - val_acc: 0.8924
Epoch 3/20
48000/48000 [==============================] - 2s 43us/step - loss: 0.4235 - acc: 0.8871 - val_loss: 0.3796 - val_acc: 0.9006
Epoch 4/20
48000/48000 [==============================] - 2s 44us/step - loss: 0.3949 - acc: 0.8925 - val_loss: 0.3599 - val_acc: 0.9034
Epoch 5/20
48000/48000 [==============================] - 2s 43us/step - loss: 0.3766 - acc: 0.8966 - val_loss: 0.3470 - val_acc: 0.9053
Epoch 6/20
48000/48000 [==============================] - 2s 44us/step - loss: 0.3635 - acc: 0.9000 - val_loss: 0.3382 - val_acc: 0.9065
Epoch 7/20
48000/48000 [==============================] - 2s 45us/step - loss: 0.3536 - acc: 0.9024 - val_loss: 0.3301 - val_acc

In [17]:
model.evaluate(X_test, y_test)

10000/10000 [==============================] - 0s 20us/step


[0.2939101231336594, 0.9189]

### Add more hidden layers to improve the performance

In [18]:
nb_epoch = 20
batch_size = 128
verbose = 1
nb_classes = 10
optimizer = SGD()
validation_split = 0.2
n_hidden = 128

In [19]:
model = Sequential(layers=[
    Dense(n_hidden, input_shape=(784,)),
    Activation('relu'),
    Dense(n_hidden),
    Activation('relu'),
    Dense(nb_classes),
    Activation('softmax')
])
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epoch, validation_split=validation_split)

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 2s 42us/step - loss: 1.5334 - acc: 0.6244 - val_loss: 0.7964 - val_acc: 0.8373
Epoch 2/20
48000/48000 [==============================] - 2s 32us/step - loss: 0.6137 - acc: 0.8525 - val_loss: 0.4594 - val_acc: 0.8827
Epoch 3/20
48000/48000 [==============================] - 2s 32us/step - loss: 0.4382 - acc: 0.8831 - val_loss: 0.3735 - val_acc: 0.8985
Epoch 4/20
48000/48000 [==============================] - 1s 31us/step - loss: 0.3759 - acc: 0.8964 - val_loss: 0.3343 - val_acc: 0.9058
Epoch 5/20
48000/48000 [==============================] - 2s 38us/step - loss: 0.3415 - acc: 0.9037 - val_loss: 0.3104 - val_acc: 0.9126
Epoch 6/20
48000/48000 [==============================] - 2s 33us/step - loss: 0.3184 - acc: 0.9099 - val_loss: 0.2924 - val_acc: 0.9175
Epoch 7/20
48000/48000 [==============================] - 2s 32us/step - loss: 0.3005 - acc: 0.9148 - val_loss: 0.2779 - val_acc

In [21]:
model.evaluate(X_test, y_test)

10000/10000 [==============================] - 0s 33us/step


[0.1923598583072424, 0.945]

### Further improving the model with dropout

In [22]:
from keras.layers.core import Dropout

In [23]:
nb_epoch = 250
batch_size = 128
nb_classes = 10
optimizer = SGD()
validation_split = 0.2
n_hidden = 128
dropout = 0.3

model = Sequential(layers=[
    Dense(n_hidden),
    Activation('relu'),
    Dropout(dropout),
    Dense(n_hidden),
    Activation('relu'),
    Dropout(dropout),
    Dense(nb_classes),
    Activation('softmax')
])

model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epoch, validation_split=validation_split)

Train on 48000 samples, validate on 12000 samples
Epoch 1/250
48000/48000 [==============================] - 2s 50us/step - loss: 1.7092 - acc: 0.4543 - val_loss: 0.9021 - val_acc: 0.8151
Epoch 2/250
48000/48000 [==============================] - 2s 42us/step - loss: 0.9238 - acc: 0.7163 - val_loss: 0.5283 - val_acc: 0.8694
Epoch 3/250
48000/48000 [==============================] - 2s 44us/step - loss: 0.7003 - acc: 0.7847 - val_loss: 0.4218 - val_acc: 0.8883
Epoch 4/250
48000/48000 [==============================] - 2s 37us/step - loss: 0.5952 - acc: 0.8193 - val_loss: 0.3675 - val_acc: 0.8991
Epoch 5/250
48000/48000 [==============================] - 2s 39us/step - loss: 0.5338 - acc: 0.8389 - val_loss: 0.3356 - val_acc: 0.9067
Epoch 6/250
48000/48000 [==============================] - 2s 45us/step - loss: 0.4955 - acc: 0.8526 - val_loss: 0.3135 - val_acc: 0.9113
Epoch 7/250
48000/48000 [==============================] - 2s 38us/step - loss: 0.4612 - acc: 0.8626 - val_loss: 0.2953 - 

In [24]:
model.evaluate(X_test, y_test)

10000/10000 [==============================] - 0s 33us/step


[0.07172030183279421, 0.9789]

### Testing different optimizers

### Increasing the number of epochs

### Increasing the size of batch computation